In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
with open('creditcard.csv') as file:
    readCSV = pd.read_csv(file, delimiter=',')
    data = readCSV.values

In [3]:
fraudData = []
normalData = []
for i in range(len(data)):
    if (data[i,30] == 1):
        fraudData.append(data[i])
    else:
        normalData.append(data[i])
fraudData = np.array(fraudData)
normalData = np.array(normalData)

In [4]:
np.random.shuffle(normalData)
normalData = data[0:492]
data = np.concatenate((normalData, fraudData), axis = 0)
np.random.shuffle(data)
answers = data[:,30]
data = data[:,:30]

In [5]:
batchSz = 10

In [6]:
U = tf.Variable(tf.random_normal([30, 900], stddev=.1))
bU = tf.Variable(tf.random_normal([900], stddev=.1))
V = tf.Variable(tf.random_normal([900, 2], stddev=.1))
bV = tf.Variable(tf.random_normal([2], stddev=.1))

inputFeatures = tf.placeholder(tf.float32, shape=[batchSz, 30])
ans = tf.placeholder(tf.int64, shape=[batchSz])

L1Output = tf.matmul(inputFeatures, U) + bU
L1Output = tf.nn.relu(L1Output)
logs = tf.matmul(L1Output, V) + bV
prbs = tf.nn.softmax(logs)
xEnt = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logs, labels=ans)
train = tf.train.AdamOptimizer(0.0001).minimize(xEnt)
numCorrect = tf.equal(tf.argmax(prbs,1), ans)
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
datalength = len(data)
numIter = 0
sumAcc = 0
for epoch in range(10):
    for i in range(int((datalength*3)/4)):
        inputs = data[i:i+batchSz]
        answr = answers[i:i+batchSz]
        i += batchSz
        sess.run(train, feed_dict={inputFeatures: inputs, ans: answr})

In [8]:
sumAcc = 0
numIter = 0
for i in range(int((datalength*3)/4), datalength-9):
    inputs = data[i:i+batchSz]
    answr = answers[i:i+batchSz]
    sumAcc += sess.run(accuracy, feed_dict={inputFeatures: inputs, ans: answr})
    i += batchSz
    numIter += 1

In [9]:
print(sumAcc/numIter)

0.987341769134
